In [19]:
import warnings
warnings.filterwarnings("ignore")


import pandas as pd
from datetime import datetime
from collections import Counter
from itertools import permutations

from tqdm import tqdm


import pymongo
import bson
username = "tagger-admin"
password = "tvaiadmin"
db_client = pymongo.MongoClient('104.198.62.226', username=username,
                                password=password,
                                authSource='tags',
                                authMechanism='SCRAM-SHA-256', port=27017).tags

In [2]:
import aerospike
config = {
    'hosts': [('35.228.136.58', 3000)]
}
client = aerospike.client(config).connect()
data_as_key = ("ids_to_validate", "data", "data")

tags_by_tag_old = client.get(data_as_key)[-1]['tags']

In [3]:
# tags_by_tag_old["cloud"]

In [4]:
import dropbox

def getImages(folder):
    processedData = []
    for image in folder:
        processedData.append(image.name)
    return processedData

def getMoreThanCan(folder, dbx):
    images = getImages(folder.entries)
    while True:
        folder = dbx.files_list_folder_continue(folder.cursor)
        images += getImages(folder.entries)
        if not folder.has_more:
            break
    return images

def getFilesLists(categories, dbx):
    files = []
    folder = dbx.files_list_folder(path=categories)
    if folder.has_more:
        data = getMoreThanCan(folder, dbx)
        files += data
        print(categories, len(data))
    else:
        data = getImages(folder.entries)
        files += data
        print(categories, len(data))
    return files


def get1dList(data): return list(itertools.chain.from_iterable(data))
    

def getProccessedData():
    
    access_token = '7wYL8EDqrSAAAAAAAAAjFKIa8fjneyf3Z6xIHPeK6N09ZyGZrVjY4RZGQPSajNY-'
    timeout = 10.0
    
    dbx = dropbox.Dropbox(access_token, timeout=timeout)
    
    foldersPaths = {
        "ALYONA" : "/graphic.one/Conveoyr/ValidatedToSearch"
    }
    
    Files = getFilesLists(foldersPaths["ALYONA"], dbx)
    FilesID = [i[:-4] for i in Files]
        
    return FilesID

data = getProccessedData()

/graphic.one/Conveoyr/ValidatedToSearch 1934


In [5]:
data

['5e2b58faefe3ebd08830c54e',
 '5e22864c258fc8e36e1dc39b',
 '5e1aff12e2bf297569fef60b',
 '5e2eb2da468372f5bf76fc1f',
 '5e2e657b5d65351045695ce8',
 '5e02825ca3c1a6a8474027e5',
 '5e414f82cf0291b2f06d5e50',
 '5e41617a7f6d48194fdfc217',
 '5e236712f35e093e3c0bd131',
 '5e2c47a9f38279f0579bdece',
 '5e03e086915d872d72f10703',
 '5e28fa50eee9bbeb941dac3f',
 '5e065588e3935fbdc9ecc996',
 '5e1a3dc7f358e886f6d1af35',
 '5e05d242e04c8c1bf8d2308b',
 '5e01fa1b37871d70d55c9651',
 '5e19c74a9d7d158ea4de4ca2',
 '5e2237ef8950e520a11668bb',
 '5e067dc6878fa1320ca7cb25',
 '5e1ce4f023bf3bc02c10f5f8',
 '5e1b6f5e20eace546b9e5804',
 '5dfba6baeefb05db161d5ec3',
 '5e414d52236cd951f58c1096',
 '5e202788b9332cbf51771b56',
 '5e253cb5b0cd4551aa3ce1ab',
 '5e024bf6a3c1a6a8474021dc',
 '5e2bda9ede1de11f269abfdf',
 '5e17f2419507e2c2963286bb',
 '5e03ba725c4421afdcf57a83',
 '5e05ef07756d6435eb6422da',
 '5e41613d0fd23bd8df46cd57',
 '5e245d60e19e8e2333fa1baa',
 '5e2f63c9a8c22f1e674c150d',
 '5dfc888a51e9d21cd70f117d',
 '5e415e232123

In [6]:
def dictImplementation(tagsList):
    result = []
    for tag in tagsList:
        if tag.lower() in tags_by_tag_old and tags_by_tag_old[tag.lower()] != [0]:
            result += tags_by_tag_old[tag.lower()]
        if tag.lower() not in tags_by_tag_old:
            result.append(tag.lower())
            print(tag.lower())
    return list(set(result))

In [52]:
def updateAPP(bson_id, new_list):
    before = db_client["tweets_pipeline_v2"].find_one({"_id":bson.ObjectId(bson_id)})
#     print(bson_id)
    print("BEFORE:", before["status"])
    date = datetime.now()
    ########################## CHANGES ######################
    status = "graphicone_search"
#     print()
#     print("status", status)
#     print("approved_timestamp", [date])
#     print("grafeed_confirmed", new_list)
#     print("additional_info", "moderator passed")
#     print("grafeed_approver", "VasiliiUshakov")
#     print()
    
    
    
    db_client["tweets_pipeline_v2"].update_one({"_id":bson.ObjectId(bson_id)},
                                    {"$set":{"status": status,
                                             "approved_timestamp": date,
                                             "grafeed_confirmed": new_list,
                                             "additional_info": "moderator tool passed",
                                             "grafeed_approver": "VasiliiUshakov",
                                            }})
    
    

    after = db_client["tweets_pipeline_v2"].find_one({"_id":bson.ObjectId(bson_id)})
    print("AFTER:", after["status"], after["approved_timestamp"], after["grafeed_confirmed"], after["additional_info"], after["grafeed_approver"])
#     print("AFTER:", new_list)
#     print()     







  4%|▍         | 102/2651 [00:20<04:54,  8.64it/s]

In [53]:
db = db_client["tweets_pipeline_v2"].find({"$or": [
                                                    {"status":"validated"}, 
                                                    ],
                                               }).sort('validated_timestamp',pymongo.DESCENDING)

counter = 0
for i in tqdm(db, total = db.count()):
    _id = str(i["_id"])
    
    if _id in data:
        counter += 1
        if counter >= 400:
            tagsList = dictImplementation(i['confirmed_after_validate'])
            if len(tagsList) != 0:
#                 print(_id)
#                 updateAPP(_id, tagsList)
#                 break
#                 print()
    else:
        pass








  0%|          | 0/2650 [00:00<?, ?it/s]






  0%|          | 1/2650 [00:00<07:25,  5.94it/s]






  4%|▍         | 102/2650 [00:00<05:02,  8.42it/s]






 39%|███▉      | 1028/2650 [00:00<02:14, 12.03it/s]

BEFORE: validated
AFTER: graphicone_search 2020-02-18 14:34:05.752000 ['macro', 'sentiment', 'gdp', 'uk'] moderator tool passed VasiliiUshakov
BEFORE: validated
AFTER: graphicone_search 2020-02-18 14:34:05.837000 ['us', 'banking', 'loan', 'etf', 'index', 'construction & engineering'] moderator tool passed VasiliiUshakov
BEFORE: validated
AFTER: graphicone_search 2020-02-18 14:34:05.921000 ['macro', 'japan', 'index'] moderator tool passed VasiliiUshakov
BEFORE: validated
AFTER: graphicone_search 2020-02-18 14:34:06.004000 ['sentiment', 'bull market'] moderator tool passed VasiliiUshakov
BEFORE: validated
AFTER: graphicone_search 2020-02-18 14:34:06.088000 ['macro', 'uk', 'bank of england', 'interest rates', 'monetary policy'] moderator tool passed VasiliiUshakov
BEFORE: validated
AFTER: graphicone_search 2020-02-18 14:34:06.172000 ['macro', 'gdp', 'government bonds', 'germany', 'interest rates', 'bonds'] moderator tool passed VasiliiUshakov
BEFORE: validated
AFTER: graphicone_search 202

AFTER: graphicone_search 2020-02-18 14:34:09.937000 ['gdp', 'nigeria', 'egypt'] moderator tool passed VasiliiUshakov
BEFORE: validated
AFTER: graphicone_search 2020-02-18 14:34:10.021000 ['climate change', 'commodities', 'oil', 'canada'] moderator tool passed VasiliiUshakov
BEFORE: validated
AFTER: graphicone_search 2020-02-18 14:34:10.104000 ['france', 'japan', 'uk', 'sweden', 'eu', 'germany', 'automotive', 'china', 'italy', 'us', 'south korea', 'global', 'education'] moderator tool passed VasiliiUshakov
BEFORE: validated
AFTER: graphicone_search 2020-02-18 14:34:10.188000 ['emerging markets', 'etf', 'volatility', 'index', 'msci'] moderator tool passed VasiliiUshakov
BEFORE: validated
AFTER: graphicone_search 2020-02-18 14:34:10.271000 ['cpi', 'inflation', 'index', 'uk'] moderator tool passed VasiliiUshakov
BEFORE: validated
AFTER: graphicone_search 2020-02-18 14:34:10.355000 ['government bonds', 'eu', 'bonds', 'interest rates'] moderator tool passed VasiliiUshakov
BEFORE: validated
A

AFTER: graphicone_search 2020-02-18 14:34:14.457000 ['funds', 'repo'] moderator tool passed VasiliiUshakov
BEFORE: validated
AFTER: graphicone_search 2020-02-18 14:34:14.540000 ['hardware', 'software', 'data', 'global'] moderator tool passed VasiliiUshakov
BEFORE: validated
AFTER: graphicone_search 2020-02-18 14:34:14.624000 ['japan', 'vietnam', 'eu', 'thailand', 'china', 'trade', 'export', 'us', 'malaysia', 'taiwan'] moderator tool passed VasiliiUshakov
BEFORE: validated
AFTER: graphicone_search 2020-02-18 14:34:14.708000 ['smartphone', 'smart device', 'us', 'digital media', 'mobile'] moderator tool passed VasiliiUshakov
BEFORE: validated
AFTER: graphicone_search 2020-02-18 14:34:14.792000 ['government bonds', 'nomura', 'hedge funds', 'futures', 'bonds'] moderator tool passed VasiliiUshakov
BEFORE: validated
AFTER: graphicone_search 2020-02-18 14:34:14.876000 ['employment', 'macro', 'utilities', 'labor', 'education', 'manufacturing', 'health', 'transportation', 'leisure', 'financials'








 39%|███▉      | 1028/2650 [00:11<02:14, 12.03it/s]






 44%|████▍     | 1162/2650 [00:11<02:02, 12.14it/s]

...

AFTER: graphicone_search 2020-02-18 14:36:08.330000 ['legal', 'us', 'china'] moderator tool passed VasiliiUshakov
BEFORE: validated
AFTER: graphicone_search 2020-02-18 14:36:08.414000 ['luxembourg', 'czech republic', 'labor', 'belgium', 'germany', 'romania', 'portugal', 'cyprus', 'austria', 'france', 'lithuania', 'greece', 'denmark', 'developed markets', 'poland', 'slovenia', 'eu', 'social', 'finland', 'bulgaria', 'sweden', 'hungary', 'slovakia', 'latvia', 'malta', 'spain', 'employment', 'uk', 'estonia', 'croatia', 'italy', 'netherlands', 'ireland'] moderator tool passed VasiliiUshakov
BEFORE: validated
AFTER: graphicone_search 2020-02-18 14:36:08.499000 ['commodities', 'soybean', 'us', 'brazil', 'agriculture', 'retail'] moderator tool passed VasiliiUshakov
BEFORE: validated









100%|█████████▉| 2641/2650 [02:03<00:00, 12.07it/s]






100%|█████████▉| 2645/2650 [02:03<00:00, 14.16it/s]

AFTER: graphicone_search 2020-02-18 14:36:08.583000 ['vehicles', 'oil', 'commodities'] moderator tool passed VasiliiUshakov
BEFORE: validated
AFTER: graphicone_search 2020-02-18 14:36:08.668000 ['legal', 'us', 'banking', 'loan', 'financials', 'credit'] moderator tool passed VasiliiUshakov
BEFORE: validated
AFTER: graphicone_search 2020-02-18 14:36:08.751000 ['germany', 'pmi', 'metals & mining', 'manufacturing', 'index'] moderator tool passed VasiliiUshakov
BEFORE: validated









100%|█████████▉| 2648/2650 [02:03<00:00, 15.08it/s]






100%|██████████| 2650/2650 [02:03<00:00, 21.39it/s]

AFTER: graphicone_search 2020-02-18 14:36:08.836000 ['japan', 'asia', 'germany', 'indonesia', 'france', 'brazil', 'canada', 'eu', 'india', 'china', 'global', 'iran', 'environment', 'norway', 'us', 'emissions', 'americas', 'north america', 'climate change', 'russia', 'ecology'] moderator tool passed VasiliiUshakov
BEFORE: validated
AFTER: graphicone_search 2020-02-18 14:36:08.920000 ['sentiment', 'bull market', 'china', 'trade', 'commodities', 'oil', 'us', 'hedge funds', 'energy', 'global', 'funds', 'saudi arabia'] moderator tool passed VasiliiUshakov
BEFORE: validated
AFTER: graphicone_search 2020-02-18 14:36:09.005000 ['politics'] moderator tool passed VasiliiUshakov


In [60]:
print(db_client["tweets_pipeline_v2"].find({"$or": [{"status":"graphicone_search"},{"status":"graphicone_feed"}],
                                     "grafeed_approver": "VasiliiUshakov", "s3_url": {"$exists": True}}).count())

print(db_client["tweets_pipeline_v2"].find({"$or": [{"status":"graphicone_search"},{"status":"graphicone_feed"}],
                                     "grafeed_approver": "VasiliiUshakov"}).count())

1466
1466


In [47]:
for i in db_client["tweets_pipeline_v2"].find({"$or": [{"status":"graphicone_search"},{"status":"graphicone_feed"}]}):
    print(i)
    print()
#     break

{'_id': ObjectId('5dc4c2565dcf6274b6a44809'), 'tweet_text': "The World's 20 Most Profitable Companies 💵\nhttps://t.co/fY7YXoAixd via @VisualCap #money #business #economy #wealth https://t.co/GPqtI7LVLY", 'hashtags': ['money', 'business', 'economy', 'wealth'], 'tweet_timestamp': datetime.datetime(2019, 10, 27, 16, 2), 'username': 'tamaramccleary', 'user_id': 2307675307, 'user_counters': {'followers_count': 306374, 'friends_count': 215979, 'listed_count': 8590}, 'tweet_counters': {'retweet_count': 11, 'favorite_count': 11}, 'img_urls': 'http://pbs.twimg.com/media/EHz92SGVUAAXDDB.jpg', 'tweet_id': 1188486009236836358, 'downloaded': True, 'download_error': False, 'img_type': 'graphic', 'graph_type': 'other-chart', 'confirmed_after_validate': ['saudi aramco', 'apple', 'icbc', 'samsung', 'china construction bank', 'jpmorgan', 'alphabet', 'bank of america', 'shelll', 'shell', 'wells fargo', 'gazprom', 'facebook', 'intel', 'exxonmobil', 'at&t', 'citi', 'toyota'], 'status': 'graphicone_feed', '

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

